## **Install Required Dependencies**

In [2]:
!pip install transformers gradio torch safetensors
!pip install gradio
!pip install torch
!pip install safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### **Import Necessary Libraries**

In [3]:
# For loading GPT-based models
from transformers import AutoModelForCausalLM, AutoTokenizer

# For creating an interactive UI
import gradio as gr

# PyTorch for deep learning computations
import torch

### **Load GPT-Neo Model**

Model: GPT-Neo 2.7B

Precision: 16-bit float

Device: Auto CPU/GPU


In [4]:
#  Function to Load GPT-Neo Model with Automatic Resource Management

def load_gpt_neo_model():
    try:
        model_name = "EleutherAI/gpt-neo-2.7B"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        return tokenizer, model
    except Exception as e:  # Prints an error message if loading fails
        print("Error loading GPT-Neo model:", e)
        raise   # Raises the exception to stop execution

### **Define GPT-Neo Response Function**

Text: Converts text to tensor and moves it to GPU

Limit: 150 tokens

Sampling: Enabled

Top-k: 50 tokens

Nucleus: Sampling


In [5]:
# Function to generate a response using GPT-Neo

def gpt_neo_response(prompt):
    try:
        inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            inputs,
            max_length=150,
            do_sample=True,
            top_k=50,
            top_p=0.92
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Converts tensor back to text
        return response
    except Exception as e:
        return f"Error generating response: {str(e)}" # Returns an error message in case of failure

### **Initialize the Model**

load: GPT-Neo

Fallback: GPT-2 tokenizer, model

In [6]:
# Attempt to initialize the GPT-Neo model

try:
    tokenizer, model = load_gpt_neo_model()
except Exception as e:
    print("Falling back to GPT-2 model due to insufficient resources.")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

### **Create a Gradio Chatbot Interface**

In [7]:
# Function to interact with the chatbot via Gradio

def chatbot(prompt):
    return gpt_neo_response(prompt)


Calls the response generation function

In [8]:
# Create a Gradio interface for the chatbot

interface = gr.Interface(
    fn=chatbot,   # Links the chatbot function to the UI
    inputs="text",
    outputs="text",
    title="GPT-Neo Chatbot",
    description="A chatbot using GPT-Neo with fallback to GPT-2.",
    theme="default"   # Uses Gradio's default theme
)


### **Launch the Gradio Chatbot**

In [ ]:
# Starts the UI, allowing sharing and debugging

interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b1e0adf8e66ca09d12.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Public URL**

A unique Gradio public link is generated (e.g., https://c1d451a72e3218ffe9.gradio.live).

This link allows you to share the chatbot interface with others over the internet.

Note: This link typically expires after 72 hours.


**Gradio Interface**
Title: "GPT-Neo Chatbot"

This is the name of the chatbot interface, set using the title parameter in the Gradio code.

Description: "A chatbot using GPT-Neo with fallback to GPT-2."

This explains the chatbot’s functionality, as defined in the description parameter.


**Input and Output Fields**

Input Field (prompt):
Users can enter a query or prompt here.

Example: "What is GPT-Neo?"

Output Field (output):
The chatbot generates and displays its response to the query here.


**Buttons**

Submit Button:
Triggers the chatbot to process the input and generate a response.

Clear Button:
Clears both the input and output fields.

Flag Button:
Allows users to flag inappropriate or incorrect responses.